# Retina Net 

En este documento se aplicará el modelo Retina Net, el cual es una mejora del algoritmo Faster RCNN. Este algoritmo descrito [aquí](https://arxiv.org/pdf/1708.02002.pdf) fué implementado usando el framework de DL Keras. Utilizaremos la implementación que se encuentra en [este repositorio](https://github.com/fizyr/keras-retinanet).

## Entrenamiento

In [4]:
train_file = path.join("..","data","dataset","train.csv")

### Ejecutar

Luego de haber instaladado __r

```
retinanet-train csv ../data/dataset/train.csv ../data/dataset/class_map.csv

```

Luego el modelo quedará en la carpeta __snapshots/__, de donde se pordrá cargar para la inferencia. (Esto toma bastante tiempo)

## Inferencia

In [11]:
#matplotlib
%matplotlib inline

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
import seaborn as sns; sns.set()
#import cv2
import os
import os.path as path
import numpy as np
import pandas as pd

from random import sample
#import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [3]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..','data', 'models', 'resnet50_BCCD.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model
try:
    model = models.convert_model(model)
except Exception:
    pass

print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0:"RBC", 1:"WBC", 2:"Platelets"}

OSError: Unable to open file (unable to open file: name = '..\data\models\resnet50_BCCD.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Inferir

In [19]:
test_file = path.join("..","data","dataset","test.csv")
test_dataset = pd.read_csv(test_file, names = ["image", "xmin", "ymin", "xmax", "ymax", "name"])

# Testing images
test_images = test_dataset.groupby("image").groups
test_images = { path.basename(image_path) : mpimg.imread(image_path) for image_path in sample(test_images.keys(), 6)}

# Resutls
inferences = {}

# Images to show results
draw_images = {image_path: np.copy(image) for image_path, image in test_images.items()}

#TODO procesar muchas imagenes en un batch
for image_name, image in test_images.items():
    # preprocess image for network
    img = preprocess_image(image)
    img, scale = resize_image(img)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img, axis=0))
    print(f"image: {image_name}\tprocessing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale
    inferences[image_name] = (boxes[0], scores[0], labels[0])



'\nfor image_name, image in test_images.items():\n    # preprocess image for network\n    img = preprocess_image(image)\n    img, scale = resize_image(image)\n\n    # process image\n    start = time.time()\n    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img, axis=0))\n    print(f"image: {image_name}\tprocessing time: ", time.time() - start)\n\n    # correct for image scale\n    boxes /= scale\n    inferences[image_name] = (boxes[0], scores[0], labels[0])\n    '

## Resultados

In [ ]:
fig, axis = plt.subplots(nrows=3, ncols=2,figsize=(15,20) )
axis = axis.flatten()

for i, image_name in enumerate(interences.keys()):
    boxes, scores, labels = interences[image_name]
    draw = draw_images[image_name]
    for box, score, label in zip(boxes, scores, labels):
        if score < 0.5:
            break
        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    axis[i].imshow(draw)
    axis[i].set_title(image_name)
    axis[i].set_xticks([])
    axis[i].set_yticks([])
